### **Avance 2 - Proyecto Final**

In [2]:
from src.db.database import DBConnection
db1 = DBConnection()

In [3]:
# Ejecutar una query con df
query = """
select FirstName as Nombre, LastName as Apellido, Gender as Genero, HireDate as FechaContratacion,
CityName as Ciudad, CountryName as Pais from employees e join cities c on e.cityid = c.cityid
join countries co on c.countryID= co.countryID LIMIT 5;
"""
df = db1.execute_query(query)
df

2025-06-04 23:38:10,939 INFO sqlalchemy.engine.Engine SELECT DATABASE()


2025-06-04 23:38:10,940 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-04 23:38:10,942 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-06-04 23:38:10,943 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-04 23:38:10,945 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-06-04 23:38:10,945 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-04 23:38:10,949 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-04 23:38:10,949 INFO sqlalchemy.engine.Engine 
select FirstName as Nombre, LastName as Apellido, Gender as Genero, HireDate as FechaContratacion,
CityName as Ciudad, CountryName as Pais from employees e join cities c on e.cityid = c.cityid
join countries co on c.countryID= co.countryID LIMIT 5;

2025-06-04 23:38:10,950 INFO sqlalchemy.engine.Engine [generated in 0.00153s] {}
2025-06-04 23:38:10,954 INFO sqlalchemy.engine.Engine ROLLBACK


,Nombre,Apellido,Genero,FechaContratacion,Ciudad,Pais
0,Nicole,Fuller,F,2011-06-20 07:15:37,New Orleans,United States
1,Christine,Palmer,F,2011-04-27 04:07:57,Fremont,United States
2,Pablo,Cline,M,2012-03-30 18:55:23,Rochester,United States
3,Darnell,Nielsen,M,2014-03-06 06:55:03,Lubbock,United States
4,Desiree,Stuart,F,2014-11-16 22:59:55,Anaheim,United States


### **Patrones de diseño**

**Patron Singleton**

In [3]:
## Verificar Patron Singleton

db1 = DBConnection()
db2 = DBConnection()

print("Es la misma instancia?", db1 is db2)
print("Es la misma base de datos?", db1.engine.url == db2.engine.url)

Es la misma instancia? True
Es la misma base de datos? True


In [15]:
# Conectarse a la base de datos
# Esto no creará una nueva conexión, sino que reutilizará la existente

db = DBConnection()

**Patron Factory**

In [50]:
## Implementación del patrón Factory

## SalesSummary

from src.design_patterns.factory import SalesSummary

query_sales = """
select SalesID, s.ProductID, ProductName, Quantity, TotalPrice, c.CustomerID, coalesce(concat( c.LastName, ", ", c.FirstName, " ", c.MiddleInitial, "."), "Sin nombre") as CustomerName,
e.EmployeeID, concat(e.LastName, ", ", e.FirstName, " ", e.MiddleInitial, ".") as EmployeeName
from sales s join products p on s.productid = p.ProductID
join customers c on s.CustomerID = c.CustomerID 
join employees e on s.SalesPersonID = e.EmployeeID
order by c.CustomerID;
"""
df_sales = db.execute_query(query_sales)
sales = [SalesSummary.from_series(row) for _, row in df_sales.iterrows()]
sales[0].__dict__

2025-06-05 00:31:34,789 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 00:31:34,790 INFO sqlalchemy.engine.Engine 
select SalesID, s.ProductID, ProductName, Quantity, TotalPrice, c.CustomerID, coalesce(concat( c.LastName, ", ", c.FirstName, " ", c.MiddleInitial, "."), "Sin nombre") as CustomerName,
e.EmployeeID, concat(e.LastName, ", ", e.FirstName, " ", e.MiddleInitial, ".") as EmployeeName
from sales s join products p on s.productid = p.ProductID
join customers c on s.CustomerID = c.CustomerID 
join employees e on s.SalesPersonID = e.EmployeeID
order by c.CustomerID;

2025-06-05 00:31:34,790 INFO sqlalchemy.engine.Engine [generated in 0.00142s] {}
2025-06-05 00:31:35,480 INFO sqlalchemy.engine.Engine ROLLBACK


{'sale_id': 2494538,
 'product_id': 405,
 'product_name': 'Pastry - Raisin Muffin - Mini',
 'quantity': 1,
 'customer_id': 1,
 'customer_name': 'Frye, Stefanie Y.',
 'employee_id': 2,
 'employee_name': 'Palmer, Christine W.',
 'total_price': Decimal('2.00')}

In [19]:
## Implementación del patrón Factory

from src.design_patterns.factory import CustomerLocationInfo

## CustomerLocationInfo
query_customer_location = """
select CustomerID, FirstName, coalesce(MiddleInitial, "") as MiddleInitial, LastName, Address, CityName, CountryName
from customers cu join cities ci on cu.cityID = ci.CityID
join countries co on ci.CountryID = co.CountryID;
"""
df_customer_location = db.execute_query(query_customer_location)
customer_locations = [CustomerLocationInfo.from_series(row) for _, row in df_customer_location.iterrows()]


2025-06-05 00:01:40,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 00:01:40,281 INFO sqlalchemy.engine.Engine 
select CustomerID, FirstName, coalesce(MiddleInitial, "") as MiddleInitial, LastName, Address, CityName, CountryName
from customers cu join cities ci on cu.cityID = ci.CityID
join countries co on ci.CountryID = co.CountryID;

2025-06-05 00:01:40,281 INFO sqlalchemy.engine.Engine [cached since 1385s ago] {}
2025-06-05 00:01:40,645 INFO sqlalchemy.engine.Engine ROLLBACK


In [20]:
## Implementación del patrón Strategy
df_sales = db1.execute_query(query_sales)

2025-06-05 00:02:39,382 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 00:02:39,383 INFO sqlalchemy.engine.Engine 
select SalesID, s.ProductID, ProductName, Quantity, TotalPrice, c.CustomerID, coalesce(concat(c.FirstName, " ", c.MiddleInitial, ". ", c.LastName), "Sin nombre") as CustomerName,
e.EmployeeID, concat(e.FirstName, " ", e.MiddleInitial, ". ", e.LastName) as EmployeeName
from sales s join products p on s.productid = p.ProductID
join customers c on s.CustomerID = c.CustomerID 
join employees e on s.SalesPersonID = e.EmployeeID
order by c.CustomerID;

2025-06-05 00:02:39,383 INFO sqlalchemy.engine.Engine [cached since 348.6s ago] {}
2025-06-05 00:02:39,866 INFO sqlalchemy.engine.Engine ROLLBACK


In [52]:
from src.design_patterns.strategy import TotalSalesByEmployee

report = TotalSalesByEmployee()
report.generate_report(df_sales,"EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,TotalVentas
17,18,"Bartlett, Warren C.",512154.00
20,21,"Brewer, Devon D.",618744.00
13,14,"Buckley, Wendi G.",406812.00
11,12,"Chen, Lindsay M.",338664.00
2,3,"Cline, Pablo Y.",83925.00
5,6,"Collins, Holly E.",171600.00
6,7,"Cook, Chadwick P.",195111.00
10,11,"Dickson, Sonya E.",308726.00
7,8,"Dyer, Julie E.",241680.00
14,15,"Finley, Kari D.",416940.00


In [53]:
from src.design_patterns.strategy import AverageSalesByEmployee

report = AverageSalesByEmployee()
report.generate_report(df_sales,"EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,Promedio de ventas
17,18,"Bartlett, Warren C.",232.269388
20,21,"Brewer, Devon D.",275.241993
13,14,"Buckley, Wendi G.",181.450491
11,12,"Chen, Lindsay M.",156.210332
2,3,"Cline, Pablo Y.",39.662098
5,6,"Collins, Holly E.",78.499543
6,7,"Cook, Chadwick P.",91.903439
10,11,"Dickson, Sonya E.",143.460037
7,8,"Dyer, Julie E.",106.326441
14,15,"Finley, Kari D.",197.414773


In [54]:
from src.design_patterns.strategy import ProductSalesByEmployee
report = ProductSalesByEmployee()
report.generate_report(df_sales, "EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,Cantidad de productos vendidos
17,18,"Bartlett, Warren C.",2205
20,21,"Brewer, Devon D.",2248
13,14,"Buckley, Wendi G.",2242
11,12,"Chen, Lindsay M.",2168
2,3,"Cline, Pablo Y.",2116
5,6,"Collins, Holly E.",2186
6,7,"Cook, Chadwick P.",2123
10,11,"Dickson, Sonya E.",2152
7,8,"Dyer, Julie E.",2273
14,15,"Finley, Kari D.",2112


In [45]:
import importlib
import src.design_patterns.strategy
importlib.reload(src.design_patterns.strategy)

<module 'src.design_patterns.strategy' from 'c:\\Users\\veron\\OneDrive\\Documentos\\curso Henry\\Proyecto-Final\\src\\design_patterns\\strategy.py'>